# Profiling RGPD - Stages (2023-2025)

Objectif: comprendre la structure du CSV, mesurer la qualite, et identifier les risques RGPD avant toute transformation.

Pourquoi c'est important:
- On ne nettoie pas une donnee qu'on ne comprend pas.
- Le profiling revele les colonnes sensibles (PII), les valeurs manquantes, et les incoherences.
- On prend des decisions de nettoyage et d'anonymisation sur des faits, pas des suppositions.


In [ ]:
import pandas as pd
from scripts.config import RAW_PATH, PII_COLUMNS


In [ ]:
# Charger le CSV brut (separateur ';', encodage latin1)
df = pd.read_csv(RAW_PATH, sep=';', encoding='latin1')
df.shape


In [ ]:
# Types et colonnes
df.dtypes


In [ ]:
# Pourcentage de valeurs manquantes
missing_pct = (df.isna().mean() * 100).round(2)
missing_pct.sort_values(ascending=False)


In [ ]:
# PII detectees selon la liste connue
pii_found = [c for c in df.columns if c in PII_COLUMNS]
pii_found


In [ ]:
# Apercu sans PII (privacy first)
non_pii_cols = [c for c in df.columns if c not in PII_COLUMNS]
df[non_pii_cols].head(3)


## Conclusion

Prochaines etapes:
1. Supprimer/anonymiser les colonnes PII des la sortie du brut.
2. Nettoyer et normaliser les colonnes texte.
3. Charger la donnee propre dans DuckDB pour construire des marts analytiques.
